In [1]:
import pymesh
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import numpy as np

In [7]:
mesh = pymesh.load_mesh("PLY/target.ply")
print(mesh.vertices)

[[  5.08071995  26.2343998   -0.58441103]
 [  4.99601984  25.92970085  -0.58441103]
 [  5.08071995  25.92970085  -0.45867801]
 ..., 
 [ -4.97396994  12.21879864  20.07859993]
 [ -4.5169301   12.56309891  21.01469994]
 [ -4.5169301   13.58989906  22.11610031]]
